![ga4](https://www.google-analytics.com/collect?v=2&tid=G-6VDTYWLKX6&cid=1&en=page_view&sid=1&dl=statmike%2Fvertex-ai-mlops%2F03+-+BigQuery+ML+%28BQML%29&dt=BQML+Feature+Engineering+-+reuseable+modular.ipynb)

# BigQuery Feature Engineering - Reusable and Modular

---
## Colab Setup

To run this notebook in Colab click [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Vertex%20AI%20GenAI%20For%20Document%20Q&A%20v2%20-%20MLB%20Rules%20For%20Baseball.ipynb) and run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    import google.colab
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

## Installs (If Needed)

The clients packages may need installing in this environment. 

In [3]:
# tuples of (import name, install name)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform'),
    ('google.cloud.documentai', 'google-cloud-bigquery'),
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [5]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

---
## Setup

Inputs

In [6]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [ ]:
REGION = 'us-central1'

# specify a GCS Bucket
GCS_BUCKET = PROJECT_ID

Packages

In [175]:
from google.cloud import bigquery
from google.cloud import aiplatform

Clients

In [9]:
# bigquery client
bq = bigquery.Client(project = PROJECT_ID)

---
## Embedded Preprocessing

### Create Model Using `TRANSFORM` statement

---
## Reusable Preprocessing

### Create Model Referencing Prior Models Preprocessing

---
## Modular Preprocessing

### Create `TRANSFORM_ONLY` Model

### Create Model Using `TRANSFORM_ONLY` Model To Preprocess Inputs

---
## Export To GCS For Complete Portability

---
## Online Serving With Vertex AI

In [ ]:
## vertex ai client
aiplatform.init(project = PROJECT_ID, location = REGION)

---
## Remove Resources Created In This Notebook

- Dataset In BigQuery
- Model Objects In BigQuery
- Model Exports in GCS
- Endpoints In Vertex AI